In [1]:
import json
import pandas as pd
import numpy as np
from copy import deepcopy
from tqdm import tqdm
import torch
from collections import defaultdict
# pd.set_option('display.max_rows', None)

In [2]:
# task3 product 데이터와 조달청 카테고리 데이터 불러오기
pro = json.load(open("./task3_product.json"))
cat = json.load(open("./zodal_cat.json"))

In [3]:
lv5_prompt = []

for i,d in enumerate(cat):
    lv5_prompt.append(d['input'])

### Lv 5 카테고리 중 중복 카테고리 제거 후 인덱싱 초기화

In [5]:
lv5_prompt2idx = {}
duple_lv5_prompt = {}

t = 0
for i, p in enumerate(lv5_prompt):
    try:
        lv5_prompt2idx[p]
        duple_lv5_prompt[i]=p
    except:
        lv5_prompt2idx[p] = t
        t += 1
        pass

In [6]:
duple_lv5_prompt

{7436: '다음 물품 분류는 주사기, 영어로 Medication or vaccine syringes 입니다. 주사기의 상세설명은 다음과 같습니다. 의약품 등을 인체에 주입하는 데에 사용하는 플라스틱 멸균 주사기. [식약처분류:A54010.01]',
 7544: '다음 물품 분류는 치과용아말감충전기, 영어로 Dental amalgam condenser 입니다. 치과용아말감충전기의 상세설명은 다음과 같습니다. 혼합된 아말감을 와동내에 압착 성형하는 기구. [식약처분류:A62040.01 치과용아말감충전기]',
 7545: '다음 물품 분류는 치과용근관플러거, 영어로 Endodontic pulp canal pluggers 입니다. 치과용근관플러거의 상세설명은 다음과 같습니다. 원주형 또는 끝이 가는 작업부분을 갖고 횡단면이 원형이며 선단이 평탄한 치과용 기구. 근관에 충전물을 주로 축방향으로 충전하는 기구. [식약처분류:A62010.04 치과용근관플러거]',
 8335: '다음 물품 분류는 의료용핸드피스, 영어로 Surgical drill handpieces 입니다. 의료용핸드피스의 상세설명은 다음과 같습니다. 인체 조직을 천자, 천공, 절삭하는 기구에 연결되는 손잡이 기구. [식약처분류:A55030.01 수동식재사용가능의료용핸드피스, A55030.02 수동식일회용의료용핸드피스, A55030.03 전동식의료용핸드피스]',
 8596: '다음 물품 분류는 의료용충전기, 영어로 Medical fillers 입니다. 의료용충전기의 상세설명은 다음과 같습니다. 인체 내부에 각종 성형 충전재 등을 압착 성형하는 기구. 치과용은 여기에서 제외됨. [식약처분류:A62010.01 의료용충전기]',
 10281: '다음 물품 분류는 에어다이브, 영어로 Air dive 입니다. 에어다이브의 상세설명은 다음과 같습니다. 위로 향하는 강한 바람 위에서 스카이다이빙 연습을 할 수 있는 장치.'}

In [8]:
lv5_idx2prompt = {}
for k, v in lv5_prompt2idx.items():
    lv5_idx2prompt[v]=k

In [10]:
drop_idx = [7436, 7544, 7545, 8335, 8596, 10281]

In [11]:
unique_prompt_idx = list(set(range(len(lv5_prompt)))-set(drop_idx))

In [14]:
new_cat = np.array(cat)[unique_prompt_idx].tolist()

In [16]:
lv5_prompt = []
lv5_key = []
lv5_prompt2idx = {}
lv5_idx2key = {}
for i,d in enumerate(new_cat):
    lv5_prompt.append(d['input'])
    lv5_key.append(d['output'].split("다음 물품 분류는 ")[-1].split(",")[0].split("입니다. ")[0])
    lv5_prompt2idx[d['input']]=i
    lv5_idx2key[i]=d['output'].split("다음 물품 분류는 ")[-1].split(",")[0].split("입니다. ")[0]

In [18]:
lv5_idx2prompt = {}
for k,v in lv5_prompt2idx.items():
    lv5_idx2prompt[v]=k

In [19]:
len(lv5_prompt2idx), len(lv5_idx2prompt), len(lv5_prompt), len(lv5_key), len(lv5_idx2key)

(21829, 21829, 21829, 21829, 21829)

In [21]:
# product 의 정답 label을 새로운 indexing 순서로 재설정
lv5_label_prompt = []
lv5_label_idx = {}
for i, d in enumerate(pro):
    lv5_label_prompt.append(d['output'])
    lv5_label_idx[i]=lv5_prompt2idx[d['output']]

### Category Embedding 중복 제거 후 재인덱싱

In [24]:
emb_zodal_cat_max_leng_512_masked = torch.load("../../../kisti_output/emb_zodal_cat_max_leng_512_masked.pt", map_location='cpu')
emb_zodal_cat_max_leng_2048_masked = torch.load("../../../kisti_output/emb_zodal_cat_max_leng_2048_masked.pt", map_location='cpu')
emb_zodal_cat_max_leng_512_eos = torch.load("../../../kisti_output/emb_zodal_cat_max_leng_512_eos.pt", map_location='cpu')
emb_zodal_cat_max_leng_2048_eos = torch.load("../../../kisti_output/emb_zodal_cat_max_leng_2048_eos.pt", map_location='cpu')

In [25]:
print(len(emb_zodal_cat_max_leng_512_masked), len(emb_zodal_cat_max_leng_2048_masked), len(emb_zodal_cat_max_leng_512_eos), len(emb_zodal_cat_max_leng_2048_eos))

21829 21835 21829 21829


In [26]:
emb_zodal_cat_max_leng_512_masked = emb_zodal_cat_max_leng_512_masked[unique_prompt_idx]
emb_zodal_cat_max_leng_2048_masked = emb_zodal_cat_max_leng_2048_masked[unique_prompt_idx]
emb_zodal_cat_max_leng_512_eos = emb_zodal_cat_max_leng_512_eos[unique_prompt_idx]
emb_zodal_cat_max_leng_2048_eos = emb_zodal_cat_max_leng_2048_eos[unique_prompt_idx]

In [27]:
len(emb_zodal_cat_max_leng_512_masked), len(emb_zodal_cat_max_leng_2048_masked), len(emb_zodal_cat_max_leng_512_eos), len(emb_zodal_cat_max_leng_2048_eos)

(21829, 21829, 21829, 21829)

In [28]:
# torch.save(emb_zodal_cat_max_leng_512_masked, "../../../kisti_output/emb_zodal_cat_max_leng_512_masked.pt")
torch.save(emb_zodal_cat_max_leng_2048_masked, "../../../kisti_output/emb_zodal_cat_max_leng_2048_masked.pt")
# torch.save(emb_zodal_cat_max_leng_512_eos, "../../../kisti_output/emb_zodal_cat_max_leng_512_eos.pt")
# torch.save(emb_zodal_cat_max_leng_2048_eos, "../../../kisti_output/emb_zodal_cat_max_leng_2048_eos.pt")

In [29]:
zs_emb_zodal_cat_max_leng_512_masked = torch.load("../../../kisti_output/zs/emb_zodal_cat_max_leng_512_masked.pt", map_location='cpu')
zs_emb_zodal_cat_max_leng_2048_masked = torch.load("../../../kisti_output/zs/emb_zodal_cat_max_leng_2048_masked.pt", map_location='cpu')
zs_emb_zodal_cat_max_leng_512_eos = torch.load("../../../kisti_output/zs/emb_zodal_cat_max_leng_512_eos.pt", map_location='cpu')
zs_emb_zodal_cat_max_leng_2048_eos = torch.load("../../../kisti_output/zs/emb_zodal_cat_max_leng_2048_eos.pt", map_location='cpu')

print(len(zs_emb_zodal_cat_max_leng_512_masked), len(zs_emb_zodal_cat_max_leng_2048_masked), len(zs_emb_zodal_cat_max_leng_512_eos), len(zs_emb_zodal_cat_max_leng_2048_eos))

21829 21829 21829 21829


In [ ]:
zs_emb_zodal_cat_max_leng_512_masked = zs_emb_zodal_cat_max_leng_512_masked[unique_prompt_idx]
zs_emb_zodal_cat_max_leng_2048_masked = zs_emb_zodal_cat_max_leng_2048_masked[unique_prompt_idx]
zs_emb_zodal_cat_max_leng_512_eos = zs_emb_zodal_cat_max_leng_512_eos[unique_prompt_idx]
zs_emb_zodal_cat_max_leng_2048_eos = zs_emb_zodal_cat_max_leng_2048_eos[unique_prompt_idx]

print(len(zs_emb_zodal_cat_max_leng_512_masked), len(zs_emb_zodal_cat_max_leng_2048_masked), len(zs_emb_zodal_cat_max_leng_512_eos), len(zs_emb_zodal_cat_max_leng_2048_eos))

In [ ]:
torch.save(zs_emb_zodal_cat_max_leng_512_masked, "../../../kisti_output/zs/emb_zodal_cat_max_leng_512_masked.pt")
torch.save(zs_emb_zodal_cat_max_leng_2048_masked, "../../../kisti_output/zs/emb_zodal_cat_max_leng_2048_masked.pt")
torch.save(zs_emb_zodal_cat_max_leng_512_eos, "../../../kisti_output/zs/emb_zodal_cat_max_leng_512_eos.pt")
torch.save(zs_emb_zodal_cat_max_leng_2048_eos, "../../../kisti_output/zs/emb_zodal_cat_max_leng_2048_eos.pt")

### Level 1-4 label 추출 후 Level 5 label과 대응.

In [ ]:
zodalraw = pd.read_csv("./zodal_raw.csv")
zodalraw = zodalraw.iloc[unique_prompt_idx].reindex()
len(zodalraw)

In [ ]:
lv4_key, lv3_key, lv2_key, lv1_key = defaultdict(list), defaultdict(list), defaultdict(list), defaultdict(list)
lv4_list, lv3_list, lv2_list, lv1_list = [], [], [], []
duple_raw = defaultdict(int)
for i, row in zodalraw.iterrows():
    duple_raw[row["context"].split("분류체계는 ")[-1].split("입니다")[0]] += 1
    
    lv4_key[row["context"].split("분류체계는 ")[-1].split("입니다")[0]].append(row["response"].split("품명: ")[-1])
    lv4_list.append(row["response"].split("품명: ")[-1])
    
    lv3_key[row["context"].split("분류체계는 ")[-1].split("입니다")[0]].append(row['response'].split("소분류: ")[-1].split(" 품명:")[0])
    lv3_list.append(row['response'].split("소분류: ")[-1].split(" 품명:")[0])
    
    lv2_key[row["context"].split("분류체계는 ")[-1].split("입니다")[0]].append(row['response'].split("중분류: ")[-1].split(" 소분류:")[0])
    lv2_list.append(row['response'].split("중분류: ")[-1].split(" 소분류:")[0])
    
    lv1_key[row["context"].split("분류체계는 ")[-1].split("입니다")[0]].append(row['response'].split("대분류: ")[-1].split(" 중분류:")[0])
    lv1_list.append(row['response'].split("대분류: ")[-1].split(" 중분류:")[0])

In [ ]:
len(lv4_key), len(lv3_key), len(lv2_key), len(lv1_key), len(zodalraw)

In [ ]:
# Lv 4
lv4_key_copy = deepcopy(lv4_key)
lv5_idx2lv4_key_tmp = {}
for i in tqdm(range(len(lv5_idx2key)), total=len(lv5_idx2key)):
    for k, v in lv4_key_copy.items():
        if lv5_idx2key[i] == k:
            if len(v) > 1:
                lv5_idx2lv4_key_tmp[i]=v.pop(0)
                lv4_key_copy[k]=v
            elif len(v) == 1:
                lv5_idx2lv4_key_tmp[i] = v[0]
            continue
            
error_list = list(set(range(21829)) - set(lv5_idx2lv4_key_tmp.keys()))

lv4_key_copy = deepcopy(lv4_key)
lv5_idx2lv4_key = {}
for i in tqdm(range(len(lv5_idx2key)), total=len(lv5_idx2key)):
    for k, v in lv4_key_copy.items():
        if lv5_idx2key[i] == k:
            if len(v) > 1:
                lv5_idx2lv4_key[i]=v.pop(0)
                lv4_key_copy[k]=v
            elif len(v) == 1:
                lv5_idx2lv4_key[i] = v[0]
            continue
        elif i in error_list:
            lv5_idx2lv4_key[i] = lv4_list[i]
            continue
            
lv4_key2idx = {}
for i, v in enumerate(list(set(lv5_idx2lv4_key.values()))):
    lv4_key2idx[v]=i
    
lv5_idx2lv4_idx = {}
for k, v in lv5_idx2lv4_key.items():
    lv5_idx2lv4_idx[k]=lv4_key2idx[v]
    
lv4_label_idx = {}
for i, v in lv5_label_idx.items():
    lv4_label_idx[i] = [lv5_idx2lv4_idx[v]]
    
with open("./lv5_idx2lv4_idx.json", "w") as f:
    json.dump(lv5_idx2lv4_idx, f)
    
with open("./lv5_idx2lv4_key.json", "w") as f:
    json.dump(lv5_idx2lv4_key, f)
    
with open("./lv4_label_idx.json", "w") as f:
    json.dump(lv4_label_idx, f)
    
with open("./lv4_key2idx.json", "w") as f:
    json.dump(lv4_key2idx, f)

In [ ]:
len(lv5_idx2lv4_idx), len(lv5_idx2lv4_key), len(lv4_label_idx), len(lv4_key2idx)

In [ ]:
# LV 3
lv3_key_copy = deepcopy(lv3_key)
lv5_idx2lv3_key_tmp = {}
for i in tqdm(range(len(lv5_idx2key)), total=len(lv5_idx2key)):
    for k, v in lv3_key_copy.items():
        if lv5_idx2key[i] == k:
            if len(v) > 1:
                lv5_idx2lv3_key_tmp[i]=v.pop(0)
                lv3_key_copy[k]=v
            elif len(v) == 1:
                lv5_idx2lv3_key_tmp[i] = v[0]
            continue
            
error_list = list(set(range(21829)) - set(lv5_idx2lv3_key_tmp.keys()))

lv3_key_copy = deepcopy(lv3_key)
lv5_idx2lv3_key = {}
for i in tqdm(range(len(lv5_idx2key)), total=len(lv5_idx2key)):
    for k, v in lv3_key_copy.items():
        if lv5_idx2key[i] == k:
            if len(v) > 1:
                lv5_idx2lv3_key[i]=v.pop(0)
                lv3_key_copy[k]=v
            elif len(v) == 1:
                lv5_idx2lv3_key[i] = v[0]
            continue
        elif i in error_list:
            lv5_idx2lv3_key[i] = lv3_list[i]
            continue
            
lv3_key2idx = {}
for i, v in enumerate(list(set(lv5_idx2lv3_key.values()))):
    lv3_key2idx[v]=i
    
lv5_idx2lv3_idx = {}
for k, v in lv5_idx2lv3_key.items():
    lv5_idx2lv3_idx[k]=lv3_key2idx[v]
    
lv3_label_idx = {}
for i, v in lv5_label_idx.items():
    lv3_label_idx[i] = [lv5_idx2lv3_idx[v]]
    
with open("./lv5_idx2lv3_idx.json", "w") as f:
    json.dump(lv5_idx2lv3_idx, f)
    
with open("./lv5_idx2lv3_key.json", "w") as f:
    json.dump(lv5_idx2lv3_key, f)
    
with open("./lv3_label_idx.json", "w") as f:
    json.dump(lv3_label_idx, f)
    
with open("./lv3_key2idx.json", "w") as f:
    json.dump(lv3_key2idx, f)

In [ ]:
len(lv5_idx2lv3_idx), len(lv5_idx2lv3_key), len(lv3_label_idx), len(lv3_key2idx)

In [ ]:
# LV 2
lv2_key_copy = deepcopy(lv2_key)
lv5_idx2lv2_key_tmp = {}
for i in tqdm(range(len(lv5_idx2key)), total=len(lv5_idx2key)):
    for k, v in lv2_key_copy.items():
        if lv5_idx2key[i] == k:
            if len(v) > 1:
                lv5_idx2lv2_key_tmp[i]=v.pop(0)
                lv2_key_copy[k]=v
            elif len(v) == 1:
                lv5_idx2lv2_key_tmp[i] = v[0]
            continue

error_list = list(set(range(21829)) - set(lv5_idx2lv2_key_tmp.keys()))

lv2_key_copy = deepcopy(lv2_key)
lv5_idx2lv2_key = {}
for i in tqdm(range(len(lv5_idx2key)), total=len(lv5_idx2key)):
    for k, v in lv2_key_copy.items():
        if lv5_idx2key[i] == k:
            if len(v) > 1:
                lv5_idx2lv2_key[i]=v.pop(0)
                lv2_key_copy[k]=v
            elif len(v) == 1:
                lv5_idx2lv2_key[i] = v[0]
            continue
        elif i in error_list:
            lv5_idx2lv2_key[i] = lv2_list[i]
            continue

lv2_key2idx = {}
for i, v in enumerate(list(set(lv5_idx2lv2_key.values()))):
    lv2_key2idx[v]=i
    
lv5_idx2lv2_idx = {}
for k, v in lv5_idx2lv2_key.items():
    lv5_idx2lv2_idx[k]=lv2_key2idx[v]
    
lv2_label_idx = {}
for i, v in lv5_label_idx.items():
    lv2_label_idx[i] = [lv5_idx2lv2_idx[v]]
    
with open("./lv5_idx2lv2_idx.json", "w") as f:
    json.dump(lv5_idx2lv2_idx, f)
    
with open("./lv5_idx2lv2_key.json", "w") as f:
    json.dump(lv5_idx2lv2_key, f)
    
with open("./lv2_label_idx.json", "w") as f:
    json.dump(lv2_label_idx, f)
    
with open("./lv2_key2idx.json", "w") as f:
    json.dump(lv2_key2idx, f)

In [ ]:
len(lv5_idx2lv2_idx), len(lv5_idx2lv2_key), len(lv2_label_idx), len(lv2_key2idx)

In [ ]:
# Lv 1
lv1_key_copy = deepcopy(lv1_key)
lv5_idx2lv1_key_tmp = {}
for i in tqdm(range(len(lv5_idx2key)), total=len(lv5_idx2key)):
    for k, v in lv1_key_copy.items():
        if lv5_idx2key[i] == k:
            if len(v) > 1:
                lv5_idx2lv1_key_tmp[i]=v.pop(0)
                lv1_key_copy[k]=v
            elif len(v) == 1:
                lv5_idx2lv1_key_tmp[i] = v[0]
            continue
            
error_list = list(set(range(21829)) - set(lv5_idx2lv1_key_tmp.keys()))

lv1_key_copy = deepcopy(lv1_key)
lv5_idx2lv1_key = {}
for i in tqdm(range(len(lv5_idx2key)), total=len(lv5_idx2key)):
    for k, v in lv1_key_copy.items():
        if lv5_idx2key[i] == k:
            if len(v) > 1:
                lv5_idx2lv1_key[i]=v.pop(0)
                lv1_key_copy[k]=v
            elif len(v) == 1:
                lv5_idx2lv1_key[i] = v[0]
            continue
        elif i in error_list:
            lv5_idx2lv1_key[i] = lv1_list[i]
            continue
            
lv1_key2idx = {}
for i, v in enumerate(list(set(lv5_idx2lv1_key.values()))):
    lv1_key2idx[v]=i
    
lv5_idx2lv1_idx = {}
for k, v in lv5_idx2lv1_key.items():
    lv5_idx2lv1_idx[k]=lv1_key2idx[v]
    
lv1_label_idx = {}
for i, v in lv5_label_idx.items():
    lv1_label_idx[i] = [lv5_idx2lv1_idx[v]]
    
with open("./lv5_idx2lv1_idx.json", "w") as f:
    json.dump(lv5_idx2lv1_idx, f)
    
with open("./lv5_idx2lv1_key.json", "w") as f:
    json.dump(lv5_idx2lv1_key, f)
    
with open("./lv1_label_idx.json", "w") as f:
    json.dump(lv1_label_idx, f)
    
with open("./lv1_key2idx.json", "w") as f:
    json.dump(lv1_key2idx, f)

In [ ]:
len(lv5_idx2lv1_idx), len(lv5_idx2lv1_key), len(lv1_label_idx), len(lv1_key2idx)

In [ ]:
lv5_label_idx = dict([(k,[v]) for k, v in lv5_label_idx.items()])

In [ ]:
with open("./lv5_label_idx.json", "w") as f:
    json.dump(lv5_label_idx, f)

### UNSPSC Level 1 데이터구축

In [ ]:
unscp_lv1 = json.load(open("./unspsc_lv1.json"))

unscp_lv1_label = {}
for i, u in enumerate(unscp_lv1):
    lab = u['output'].split("다음 물품 분류는 ")[-1].split("입니다.")[0]
    try:
        unscp_lv1_label[lab] = lv1_key2idx[lab]
    except:
        print(i, lab)

In [ ]:
len(unscp_lv1)

In [ ]:
_un_lv1_idx = list(set(range(55))-set([37, 54]))

In [ ]:
new_unscp_lv1 = []
_lv1_un_idx = []
for t, x in enumerate(unscp_lv1):
    if t in _un_lv1_idx:
        _lv1_un_idx.append(t)
        new_unscp_lv1.append(x)

In [ ]:
len(new_unscp_lv1)

In [ ]:
with open('./new_unscp_lv1.json', 'w') as f:
    json.dump(new_unscp_lv1, f)

### 국문 조달청 카테고리 데이터 Level 1 데이터 구축

In [ ]:
zodal_lv1 = json.load(open('zodal_lv1.json'))
new_zodal_lv1 = []

zodal_lv1_label = {}
for i, u in enumerate(zodal_lv1):
    lab = u['output'].split("다음 물품 분류는 ")[-1].split("입니다.")[0]
    try:
        zodal_lv1_label[lab] = lv1_key2idx[lab]
        new_zodal_lv1.append(u)
    except:
        print(i, lab)

In [ ]:
_zodal_lv1_idx = list(set(range(56))-set([37, 54]))

In [ ]:
with open("./new_zodal_lv1.json", "w") as f:
    json.dump(new_zodal_lv1, f)

### 국문 조달청 Level 1카테고리 데이터와 UNSPSC Level 1 데이터를 Level 1 label에 대응

In [ ]:
zodal_lv1idx2lv1idx={}
for i, v in enumerate(zodal_lv1_label.values()):
    zodal_lv1idx2lv1idx[i]=v

In [ ]:
unscp_lv1idx2lv1idx={}
for i, v in enumerate(unscp_lv1_label.values()):
    unscp_lv1idx2lv1idx[i]=v

In [ ]:
with open("./zodal_lv1idx2lv1idx.json", "w") as f:
    json.dump(zodal_lv1idx2lv1idx, f)
    
with open("./unscp_lv1idx2lv1idx.json", "w") as f:
    json.dump(unscp_lv1idx2lv1idx, f)

In [ ]:
lv1key2lv5idx = defaultdict(list)
lv1idx2lv5idx = defaultdict(list)

for k, v in lv5_idx2lv1_key.items():
    lv1key2lv5idx[v].append(k)
    lv1idx2lv5idx[lv1_key2idx[v]].append(k)

In [ ]:
with open("./lv1idx2lv5idx.json", "w") as f:
    json.dump(lv1idx2lv5idx, f)

### 국문 조달청 Level 1 카테고리 label 과 임베딩 index 매칭

In [ ]:
emb_zodal_cat_lv1_max_leng_512_eos = torch.load("../../../kisti_output/emb_zodal_cat_lv1_max_leng_512_eos.pt", map_location='cpu')
emb_zodal_cat_lv1_max_leng_512_masked = torch.load("../../../kisti_output/emb_zodal_cat_lv1_max_leng_512_masked.pt", map_location='cpu')
emb_zodal_cat_lv1_max_leng_2048_eos = torch.load("../../../kisti_output/emb_zodal_cat_lv1_max_leng_2048_eos.pt", map_location='cpu')
emb_zodal_cat_lv1_max_leng_2048_masked = torch.load("../../../kisti_output/emb_zodal_cat_lv1_max_leng_2048_masked.pt", map_location='cpu')

_lv1_idx = list(set(range(56))-set([37, 54]))

emb_zodal_cat_lv1_max_leng_512_eos=emb_zodal_cat_lv1_max_leng_512_eos[_lv1_idx]
emb_zodal_cat_lv1_max_leng_512_masked=emb_zodal_cat_lv1_max_leng_512_masked[_lv1_idx]
emb_zodal_cat_lv1_max_leng_2048_eos=emb_zodal_cat_lv1_max_leng_2048_eos[_lv1_idx]
emb_zodal_cat_lv1_max_leng_2048_masked=emb_zodal_cat_lv1_max_leng_2048_masked[_lv1_idx]

torch.save(emb_zodal_cat_lv1_max_leng_512_eos, "../../../kisti_output/emb_zodal_cat_lv1_max_leng_512_eos.pt")
torch.save(emb_zodal_cat_lv1_max_leng_512_masked, "../../../kisti_output/emb_zodal_cat_lv1_max_leng_512_masked.pt")
torch.save(emb_zodal_cat_lv1_max_leng_2048_eos, "../../../kisti_output/emb_zodal_cat_lv1_max_leng_2048_eos.pt")
torch.save(emb_zodal_cat_lv1_max_leng_2048_masked, "../../../kisti_output/emb_zodal_cat_lv1_max_leng_2048_masked.pt")

### UNSPSC Level 1 label과 임베딩 index 매칭

In [ ]:
emb_unspsc_lv1_max_leng_512_eos = torch.load("../../../kisti_output/emb_unspsc_lv1_max_leng_512_eos.pt", map_location='cpu')
emb_unspsc_lv1_max_leng_512_masked = torch.load("../../../kisti_output/emb_unspsc_lv1_max_leng_512_masked.pt", map_location='cpu')
emb_unspsc_lv1_max_leng_2048_eos = torch.load("../../../kisti_output/emb_unspsc_lv1_max_leng_2048_eos.pt", map_location='cpu')
emb_unspsc_lv1_max_leng_2048_masked = torch.load("../../../kisti_output/emb_unspsc_lv1_max_leng_2048_masked.pt", map_location='cpu')

emb_unspsc_lv1_max_leng_512_eos=emb_unspsc_lv1_max_leng_512_eos[_un_lv1_idx]
emb_unspsc_lv1_max_leng_512_masked=emb_unspsc_lv1_max_leng_512_masked[_un_lv1_idx]
emb_unspsc_lv1_max_leng_2048_eos=emb_unspsc_lv1_max_leng_2048_eos[_un_lv1_idx]
emb_unspsc_lv1_max_leng_2048_masked=emb_unspsc_lv1_max_leng_2048_masked[_un_lv1_idx]

torch.save(emb_unspsc_lv1_max_leng_512_eos, "../../../kisti_output/emb_unspsc_lv1_max_leng_512_eos.pt")
torch.save(emb_unspsc_lv1_max_leng_512_masked, "../../../kisti_output/emb_unspsc_lv1_max_leng_512_masked.pt")
torch.save(emb_unspsc_lv1_max_leng_2048_eos, "../../../kisti_output/emb_unspsc_lv1_max_leng_2048_eos.pt")
torch.save(emb_unspsc_lv1_max_leng_2048_masked, "../../../kisti_output/emb_unspsc_lv1_max_leng_2048_masked.pt")

In [ ]:
len(emb_unspsc_lv1_max_leng_512_eos), len(emb_unspsc_lv1_max_leng_512_masked), len(emb_unspsc_lv1_max_leng_2048_eos), len(emb_unspsc_lv1_max_leng_2048_masked)